In [1]:
!nvidia-smi

Tue Mar 08 21:51:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   28C    P8    11W / 260W |    319MiB / 11264MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [4]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:0')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=0))

In [6]:
torch.cuda.device_count()

1

In [7]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [8]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [9]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [10]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.6293, 0.1621, 0.3503],
        [0.9056, 0.3274, 0.8836]])

In [12]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [14]:
X + Z

tensor([[2., 2., 2.],
        [2., 2., 2.]], device='cuda:0')

In [16]:
Z.cuda(0) is Z

True

In [17]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [18]:
net(X)

tensor([[-0.7107],
        [-0.7107]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [19]:
net[0].weight.data.device

device(type='cuda', index=0)